# 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)
LDA는 단어의 순서는 신경쓰지 않음. <br>

문서1 : 저는 사과랑 바나나를 먹어요<br>
문서2 : 우리는 귀여운 강아지가 좋아요<br>
문서3 : 저의 깜찍하고 귀여운 강아지가 바나나를 먹어요<br>

<각 문서의 토픽 분포><br>
문서1 : 토픽 A 100%<br>
문서2 : 토픽 B 100%<br>
문서3 : 토픽 B 60%, 토픽 A 40%<br>

<각 토픽의 단어 분포><br>
토픽A : 사과 20%, 바나나 40%, 먹어요 40%, 귀여운 0%, 강아지 0%, 깜찍하고 0%, 좋아요 0%<br>
토픽B : 사과 0%, 바나나 0%, 먹어요 0%, 귀여운 33%, 강아지 33%, 깜찍하고 16%, 좋아요 16%<br>


각각의 문서는 다음의 과정을 거쳐 작성되었다고 가정: <br>
<ol>
<li>문서에 사용할 단어의 개수 N을 정함 <br>ex-5개</li>
<li>문서에 사용할 토픽의 혼합을 확률 분포에 기반하여 결정.<br>ex-토픽이 2개인 경우, 강아지-60%, 과일-40%와 같이 선택</li>
<li>문서에 사용할 단어를 정함<br>(1)토픽 분포에서 토픽 T를 확률적으로 선택<br>ex-강아지 토픽 선택-60%, 과일 토픽 선택-40%<br>(2)선택한 토픽 T에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어 선택<br>ex-강아지 토픽을 선택했다면, 33% 확률로 강아지라는 단어 선택</li>
</ol>
<h4> LDA는 토픽을 뽑기 위해 위 과정을 역추적함.</h4>

## LDA 수행 과정
1) 사용자는 LDA 알고리즘에게 토픽의 개수 k를 알려줌.<br>
2) LDA는 모든 단어를 k개 중 하나의 토픽에 랜덤으로 할당함. (랜덤이므로 전부 틀린 상태) <br>
3) 모든 문서의 모든 단어에 대해 아래 사항 반복<br>
&nbsp;&nbsp;&nbsp;&nbsp;어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있는 상태라고 가정. <br>&nbsp;&nbsp;&nbsp;&nbsp;이에따라 단어 w는 두 기준에 따라 토픽을 재할당함.
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(1) $P(topic \ t | document \ d): \text{문서 d의 단어들 중 토픽 t에 해당하는 단어들의 비율}$
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(2) $P(word \ w | topic \ t): \text{전체 문서에서의 각 토픽들 t에서 해당 단어 w의 분포}$

## 잠재 디리클레 할당 vs 잠재 의미분석
LDA: 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하려 토픽을 추출.<br>
LSA: DTM을 차원 축소하여 축소된 차원에서 근접 단어들을 토픽으로 묶음.

## Gensim 이용 실습

### 정수 인코딩 + 단어 집합 만들기

In [1]:
# 6-1과 동일한 전처리
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

news_df = pd.DataFrame({'document':documents}) # DF 생성
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ") # 알파벳 제외 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
tokenized_doc

<ipython-input-1-6520c178866e>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ") # 알파벳 제외 제거


0        [well, sure, story, seem, biased, disagree, st...
1        [yeah, expect, people, read, actually, accept,...
2        [although, realize, principle, strongest, poin...
3        [notwithstanding, legitimate, fuss, proposal, ...
4        [well, change, scoring, playoff, pool, unfortu...
                               ...                        
11309    [danny, rubenstein, israeli, journalist, speak...
11310                                                   []
11311    [agree, home, runs, clemens, always, memorable...
11312    [used, deskjet, orange, micros, grappler, syst...
11313    [argument, murphy, scared, hell, came, last, y...
Name: clean_doc, Length: 11314, dtype: object

In [4]:
# 각 단어에 정수 인코딩 + 각 뉴스에서의 단어의 빈도수 기록
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
corpus[1]

[(52, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 1),
 (65, 1),
 (66, 2),
 (67, 1),
 (68, 1),
 (69, 1),
 (70, 1),
 (71, 2),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 1),
 (76, 1),
 (77, 1),
 (78, 2),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 2),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1)]

In [5]:
dictionary[66] # 정수 인코딩이 66으로 할당된 단어가 2번째 뉴스에서 2번 등장. 해당 단어 dictionary에서 찾기

'faith'

In [6]:
# 총 학습된 단어 개수
len(dictionary)

64281

### LDA 모델 훈련시키기

In [7]:
import gensim
NUM_TOPICS = 20 # 20개의 토픽
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics = NUM_TOPICS, 
                                           id2word=dictionary, 
                                           passes=15) # 알고리즘의 동작 횟수
topics = ldamodel.print_topics(num_words=4) # 출력할 단어 수

for topic in topics:
    print(topic) # 토픽, 토픽별 기여도 * 단어

(0, '0.020*"armenian" + 0.017*"armenians" + 0.014*"turkish" + 0.012*"said"')
(1, '0.010*"thanks" + 0.010*"system" + 0.010*"windows" + 0.009*"drive"')
(2, '0.010*"baltimore" + 0.009*"decenso" + 0.009*"forsale" + 0.008*"tyre"')
(3, '0.017*"scsi" + 0.013*"power" + 0.010*"char" + 0.010*"chip"')
(4, '0.018*"space" + 0.007*"information" + 0.007*"nasa" + 0.007*"university"')
(5, '0.025*"game" + 0.022*"team" + 0.018*"games" + 0.016*"year"')
(6, '0.011*"jesus" + 0.007*"people" + 0.007*"believe" + 0.007*"christian"')
(7, '0.011*"nist" + 0.009*"ncsl" + 0.009*"vram" + 0.008*"mask"')
(8, '0.024*"file" + 0.013*"program" + 0.011*"window" + 0.010*"output"')
(9, '0.009*"good" + 0.007*"bike" + 0.005*"much" + 0.005*"engine"')
(10, '0.014*"navy" + 0.014*"picture" + 0.011*"sleeve" + 0.011*"naval"')
(11, '0.018*"smokeless" + 0.017*"kent" + 0.013*"cheers" + 0.008*"males"')
(12, '0.010*"pixmap" + 0.010*"cross" + 0.010*"linked" + 0.009*"doug"')
(13, '0.020*"radar" + 0.017*"cars" + 0.016*"ford" + 0.015*"master"

### LDA 시각화

In [19]:
#!pip install pyLDAvis

/home/yeon_28/anaconda3/envs/nlp/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

/home/yeon_28/anaconda3/envs/nlp/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 문서 별 토픽 분포

In [15]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i, '번째 문서의 topic 비율은', topic_list)

0 번째 문서의 topic 비율은 [(0, 0.0785168), (14, 0.36305934), (15, 0.18091872), (17, 0.33053002), (18, 0.03486096)]
1 번째 문서의 topic 비율은 [(6, 0.27453232), (9, 0.178855), (14, 0.4742003), (15, 0.05133943)]
2 번째 문서의 topic 비율은 [(14, 0.51015437), (15, 0.018433023), (17, 0.45767522)]
3 번째 문서의 topic 비율은 [(8, 0.07575503), (12, 0.019054566), (14, 0.47246364), (15, 0.073501796), (16, 0.34747148)]
4 번째 문서의 topic 비율은 [(2, 0.25104427), (5, 0.23551998), (8, 0.052010864), (14, 0.43178275)]


/home/yeon_28/anaconda3/envs/nlp/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
# dataframe 형식으로 변환
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()
    
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True) # 각 문서에 대해 비중이 높은 토픽순으로 토픽 정렬
        
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽, 가장 비중이 높은 토픽의 비중, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호 칼럼
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable.head(10)

/home/yeon_28/anaconda3/envs/nlp/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,14.0,0.3631,"[(0, 0.07851146), (14, 0.36306205), (15, 0.180..."
1,1,14.0,0.4743,"[(6, 0.27449432), (9, 0.17884265), (14, 0.4742..."
2,2,14.0,0.5101,"[(14, 0.5101485), (15, 0.018432967), (17, 0.45..."
3,3,14.0,0.4724,"[(8, 0.07576392), (12, 0.019054662), (14, 0.47..."
4,4,14.0,0.4318,"[(2, 0.25104278), (5, 0.23554178), (8, 0.05194..."
5,5,6.0,0.3444,"[(0, 0.06650265), (5, 0.13871133), (6, 0.34440..."
6,6,1.0,0.3144,"[(1, 0.31439158), (3, 0.034627195), (6, 0.0555..."
7,7,14.0,0.6993,"[(3, 0.034566518), (14, 0.699287), (17, 0.2521..."
8,8,14.0,0.5874,"[(0, 0.030635834), (1, 0.18813676), (2, 0.0515..."
9,9,14.0,0.6895,"[(0, 0.018579956), (9, 0.23224328), (14, 0.689..."
